# Project Setup File for arXiv Semantic Search App

## Install required dependencies

In [1]:
# %%capture
!pip install tensorflow tensorflow-hub chromadb psycopg psycopg2-binary pgvector kagglehub streamlit numpy pandas matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
  Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 983.6 kB/s eta 0:00:000:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.3 MB/s eta 0:00:0000:0100:01


## Add PostgreSQL Connection Setup

Do not forget, you need to setup **pgvector** beforehand for this code to function.

In [1]:
# PostgreSQL setup
PG_USER = "postgres"
PG_PASSWORD = "password"  # adjust as needed
PG_HOST = "localhost"
PG_PORT = 5434  # specify your custom port here
PG_DBNAME = "arxivdb"

## Data Preperation

### Download the Newest arXiv Dataset from Kaggle 

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("Cornell-University/arxiv")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Nima\.cache\kagglehub\datasets\Cornell-University\arxiv\versions\210


### Load Data to Python

You can either load your desired number of articles:

In [4]:
# Load a small subset of ArXiv metadata for demonstration
import json


DATA_FILE = f"{path}/arxiv-metadata-oai-snapshot.json"

# Let's just read a few records to keep it manageable
docs = []
max_docs = 200000 # adjust this number as you wish
with open(DATA_FILE, 'r') as f:
    for i, line in enumerate(f):
        if i >= max_docs:
            break
        data = json.loads(line)
        # Each data item is a dictionary with keys like 'title' and 'abstract'
        if 'title' in data and 'abstract' in data:
            docs.append({
                'id': data['id'],
                'title': data['title'],
                'abstract': data['abstract']
            })

print(f"Loaded {len(docs)} documents.")


Loaded 200000 documents.


Or load all of them:

In [2]:
from tqdm.notebook import tqdm  # Use tqdm.auto if preferred
import json

DATA_FILE = "/home/nima/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/210/arxiv-metadata-oai-snapshot.json"

docs = []


def get_total_lines(filename):
    with open(filename, 'rb') as f:
        return sum(buf.count(b'\n') for buf in iter(lambda: f.read(1024*1024), b''))

total_lines = get_total_lines(DATA_FILE)
print(f"Total lines in the dataset: {total_lines}")

# Loading without knowing the total number of lines
with open(DATA_FILE, 'r') as f:
    for line in tqdm(f, desc="Loading ArXiv documents"):
        data = json.loads(line)
        if 'title' in data and 'abstract' in data:
            docs.append({
                'id': data['id'],
                'title': data['title'],
                'abstract': data['abstract']
            })

print(f"Loaded {len(docs)} documents.")


Total lines in the dataset: 2626136


Loading ArXiv documents: 0it [00:00, ?it/s]

Loaded 2626136 documents.


## Load the Embedding Model (Google Universal Sentence Encoder)

If you do not have CUDA setup you can force TensorFlow to use CPU by uncommenting the related code.

In [5]:
import os
import tensorflow_hub as hub

# Force TensorFlow to use the CPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Load the Universal Sentence Encoder model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# To be embedded
texts = [d['title'] + " " + d['abstract'] for d in docs]

## ChromaDB and PGVector

### Connection and Batch Processing the Embedded Data

In [7]:
import tensorflow_hub as hub
import numpy as np
import os
from tqdm.notebook import tqdm
import time
import chromadb
from chromadb.config import Settings
import psycopg
import pgvector.psycopg

# ChromaDB setup
chroma_client = chromadb.PersistentClient(path="./chromadb_arxiv")
try:
    chroma_client.delete_collection("arxiv_collection")
except:
    pass
collection = chroma_client.create_collection("arxiv_collection")

# Connect to the default 'postgres' database to check for 'arxivdb'
with psycopg.connect(
    user=PG_USER,
    password=PG_PASSWORD,
    host=PG_HOST,
    port=PG_PORT,
    dbname="postgres",  # connect to the default database
    autocommit=True
) as conn:
    with conn.cursor() as cur:
        # Check if 'arxivdb' exists
        cur.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (PG_DBNAME,))
        exists = cur.fetchone()

        # Create 'arxivdb' if it doesn't exist
        if not exists:
            cur.execute(f"CREATE DATABASE {PG_DBNAME};")
            print(f"Database '{PG_DBNAME}' created.")
        else:
            print(f"Database '{PG_DBNAME}' already exists.")

# Connect to the 'arxivdb' database
with psycopg.connect(
    user=PG_USER,
    password=PG_PASSWORD,
    host=PG_HOST,
    port=PG_PORT,
    dbname=PG_DBNAME,
    autocommit=True
) as conn:
    with conn.cursor() as cur:
        # Create the pgvector extension if it doesn't exist
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        print("pgvector extension is ensured to be available.")

    # Register the vector type with the connection
    pgvector.psycopg.register_vector(conn)

    with conn.cursor() as cur:
        # Drop and recreate the 'arxiv_items' table
        cur.execute("DROP TABLE IF EXISTS arxiv_items;")
        cur.execute("CREATE TABLE arxiv_items (id TEXT PRIMARY KEY, title TEXT, abstract TEXT, embedding vector(512));")

        # Process and store embeddings in batches
        batch_size = 1000  # Adjust this depending on your system's memory

        for i in tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
            # Prepare batch data
            batch_texts = texts[i:i+batch_size]
            batch_docs = docs[i:i+batch_size]
            batch_embeddings = embed(batch_texts).numpy()

            # Insert into ChromaDB
            collection.add(
                ids=[doc['id'] for doc in batch_docs],
                embeddings=batch_embeddings.tolist(),
                metadatas=[{'title': doc['title'], 'abstract': doc['abstract']} for doc in batch_docs],
                documents=[doc['abstract'] for doc in batch_docs]
            )

            # Insert into PostgreSQL
            for doc, emb in zip(batch_docs, batch_embeddings):
                cur.execute(
                    "INSERT INTO arxiv_items (id, title, abstract, embedding) VALUES (%s, %s, %s, %s);",
                    (doc['id'], doc['title'], doc['abstract'], emb)
                )

        # Create vector index for cosine similarity in PostgreSQL
        cur.execute("CREATE INDEX arxiv_items_embedding_idx ON arxiv_items USING hnsw (embedding vector_cosine_ops);")

print("Batch processing complete! Data stored in both ChromaDB and PostgreSQL.")


Database 'arxivdb' already exists.
pgvector extension is ensured to be available.


Processing batches:   0%|          | 0/200 [00:00<?, ?it/s]

Batch processing complete! Data stored in both ChromaDB and PostgreSQL.


### Check for Missing IDs and Loading them (if there was an error with your batch processing)

In [9]:
import chromadb
from chromadb.config import Settings
import psycopg
import pgvector.psycopg

# Function to get ChromaDB collection
def get_chroma_collection():
    client = chromadb.PersistentClient(path="./chromadb_arxiv")
    try:
        return client.get_collection("arxiv_collection")
    except:
        # If collection does not exist, create it
        return client.create_collection("arxiv_collection")

# Function to connect to PostgreSQL
def get_pg_connection():
    conn = psycopg.connect(user=PG_USER, password=PG_PASSWORD, host=PG_HOST, port=PG_PORT,dbname=PG_DBNAME, autocommit=True)
    pgvector.psycopg.register_vector(conn)
    return conn

# Fetch IDs from databases
def get_existing_ids():
    # Fetch IDs from ChromaDB
    collection = get_chroma_collection()
    
    # Retrieve all data and extract IDs
    chroma_data = collection.get()
    existing_chromadb_ids = set(chroma_data["ids"])

    # Fetch IDs from PostgreSQL
    conn = get_pg_connection()
    cur = conn.cursor()
    cur.execute("SELECT id FROM arxiv_items;")
    existing_postgres_ids = {row[0] for row in cur.fetchall()}
    conn.close()

    return existing_chromadb_ids, existing_postgres_ids


In [ ]:
existing_chromadb_ids, existing_postgres_ids = get_existing_ids()
print(f"Number of IDs in ChromaDB: {len(existing_chromadb_ids)}")
print(f"Number of IDs in PostgreSQL: {len(existing_postgres_ids)}")

Number of IDs in ChromaDB: 200
Number of IDs in PostgreSQL: 200


: 

In [6]:
# Step 1: Identify Missing IDs
dataset_ids = {doc['id'] for doc in docs}  # Assuming `docs` contains your dataset
missing_ids = dataset_ids - existing_chromadb_ids - existing_postgres_ids
len(missing_ids)  # Number of missing IDs

# Step 2: Filter the Dataset for Missing IDs
missing_docs = [doc for doc in docs if doc['id'] in missing_ids]
missing_texts = [doc['title'] + " " + doc['abstract'] for doc in missing_docs]

### Saving the Results of Missing to JSON (Checkpoint in case of crash due to low RAM) 

In [8]:
import json

# Save missing_docs to a JSON file
with open("missing_docs.json", "w", encoding="utf-8") as f:
    json.dump(missing_docs, f, ensure_ascii=False, indent=4)

print("Missing documents saved to 'missing_docs.json'")

Missing documents saved to 'missing_docs.json'


In [8]:
# Load missing_docs from the JSON file
with open("missing_docs.json", "r", encoding="utf-8") as f:
    missing_docs = json.load(f)

print(f"Loaded {len(missing_docs)} missing documents.")
missing_texts = [doc['title'] + " " + doc['abstract'] for doc in missing_docs]

Loaded 96120 missing documents.


### Insert Missing Rows (Optional)

In [2]:
import chromadb
from chromadb.config import Settings
import psycopg
import pgvector.psycopg

# Function to get ChromaDB collection
def get_chroma_collection():
    client = chromadb.PersistentClient(path="./chromadb_arxiv")
    try:
        return client.get_collection("arxiv_collection")
    except:
        # If collection does not exist, create it
        return client.create_collection("arxiv_collection")

# Function to connect to PostgreSQL
def get_pg_connection():
    conn = psycopg.connect(user=PG_USER, password=PG_PASSWORD, host=PG_HOST, dbname=PG_DBNAME, autocommit=True)
    pgvector.psycopg.register_vector(conn)
    return conn

# Insert missing items into ChromaDB and PostgreSQL
def insert_missing_items(missing_docs, embed, batch_size=500):
    # Connect to databases
    collection = get_chroma_collection()
    conn = get_pg_connection()

    # Process and insert in batches
    for i in tqdm(range(0, len(missing_texts), batch_size), desc="Inserting missing items"):
        batch_texts = missing_texts[i:i + batch_size]
        batch_docs = missing_docs[i:i + batch_size]
        batch_embeddings = embed(batch_texts).numpy()

        # Insert into ChromaDB
        collection.add(
            ids=[doc['id'] for doc in batch_docs],
            embeddings=batch_embeddings.tolist(),
            metadatas=[{'title': doc['title'], 'abstract': doc['abstract']} for doc in batch_docs],
            documents=[doc['abstract'] for doc in batch_docs]
        )

        # Insert into PostgreSQL
        cur = conn.cursor()
        for doc, emb in zip(batch_docs, batch_embeddings):
            cur.execute(
                "INSERT INTO arxiv_items (id, title, abstract, embedding) VALUES (%s, %s, %s, %s);",
                (doc['id'], doc['title'], doc['abstract'], emb)
            )

    # Close PostgreSQL connection
    conn.close()
    print("Insertion of missing items completed.")

insert_missing_items(missing_docs, embed)

### Create PGVector Indexes (HNSW)

In [4]:
def create_hnsw_index(conn):
    try:
        cur = conn.cursor()
        cur.execute("""
            DO $$
            BEGIN
                IF NOT EXISTS (SELECT 1 FROM pg_indexes WHERE tablename = 'arxiv_items' AND indexname = 'embedding_hnsw_idx') THEN
                    CREATE INDEX embedding_hnsw_idx
                    ON arxiv_items USING hnsw (embedding vector_hnsw_ops);
                END IF;
            END $$;
        """)
        cur.close()
        print("HNSW index created successfully (or already exists).")
    except Exception as e:
        print(f"Failed to create HNSW index: {e}")


pg_conn = get_pg_connection()
create_hnsw_index(pg_conn)

Failed to create HNSW index: operator class "vector_hnsw_ops" does not exist for access method "hnsw"
CONTEXT:  SQL statement "CREATE INDEX embedding_hnsw_idx
                    ON arxiv_items USING hnsw (embedding vector_hnsw_ops)"
PL/pgSQL function inline_code_block line 4 at SQL statement


## Create the Streamlit App: app.py

In [8]:
%%writefile app.py
import streamlit as st
import psycopg
import chromadb
import tensorflow_hub as hub
import pgvector.psycopg
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

# Load USE once and cache
@st.cache_resource
def load_use_model():
    return hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

@st.cache_resource
def get_chroma_collection():
    client = chromadb.PersistentClient(path="./chromadb_arxiv")
    return client.get_collection("arxiv_collection")

@st.cache_resource
def get_pg_connection():
    PG_USER = "postgres"
    PG_PASSWORD = "passkey"  # adjust accordingly
    PG_HOST = "localhost"
    PG_DBNAME = "arxivdb"
    conn = psycopg.connect(user=PG_USER, password=PG_PASSWORD, host=PG_HOST, dbname=PG_DBNAME, autocommit=True)
    pgvector.psycopg.register_vector(conn)
    return conn

def search_chroma(query_embedding, top_k=5, coll=None):
    start = time.monotonic()
    results = coll.query(query_embeddings=[query_embedding], n_results=top_k)
    latency = max(0, time.monotonic() - start)
    hits = [
        {
            'id': results['ids'][0][i],
            'abstract': results['documents'][0][i],
            'title': results['metadatas'][0][i]['title']
        }
        for i in range(len(results['ids'][0]))
    ]
    return hits, latency

def search_pgvector(query_embedding, top_k=5, conn=None):
    cur = conn.cursor()
    start = time.monotonic()
    cur.execute("SELECT id, title, abstract FROM arxiv_items ORDER BY embedding <=> %s LIMIT %s;", (query_embedding, top_k))
    rows = cur.fetchall()
    latency = max(0, time.monotonic() - start)
    hits = [{'id': r[0], 'title': r[1], 'abstract': r[2]} for r in rows]
    return hits, latency


# Establish connections as soon as the app starts
use_model = load_use_model()
chroma_coll = get_chroma_collection()
pg_conn = get_pg_connection()

st.title("ArXiv Semantic Search: Performance Comparison")

# Sidebar options
db_option = st.sidebar.selectbox("Select vector DB:", ["ChromaDB", "PGVector"])
query = st.text_input("Enter your query:", "")
top_k = st.sidebar.slider("Number of results (k):", min_value=1, max_value=20, value=5)

# Maintain stateful metrics
if 'chroma_query_times' not in st.session_state:
    st.session_state.chroma_query_times = []
if 'pgvector_query_times' not in st.session_state:
    st.session_state.pgvector_query_times = []

# Placeholder for recall metrics (if labels are available)
if 'recall_chroma' not in st.session_state:
    st.session_state.recall_chroma = []
if 'recall_pgvector' not in st.session_state:
    st.session_state.recall_pgvector = []

# Search Section
st.header("Single Query Search")

if st.button("Search"):
    if query.strip():
        q_emb = use_model([query]).numpy()[0]

        if db_option == "ChromaDB":
            results, latency = search_chroma(q_emb, top_k=top_k, coll=chroma_coll)
            st.session_state.chroma_query_times.append(latency)
            # Placeholder for recall calculation
            # If labels are available, compute recall here
            # For example:
            # recall = compute_recall(true_relevant_docs, results)
            # st.session_state.recall_chroma.append(recall)
        else:
            results, latency = search_pgvector(q_emb, top_k=top_k, conn=pg_conn)
            st.session_state.pgvector_query_times.append(latency)
            # Placeholder for recall calculation
            # If labels are available, compute recall here
            # recall = compute_recall(true_relevant_docs, results)
            # st.session_state.recall_pgvector.append(recall)

        st.write(f"**Results from {db_option}** (Query took {latency:.4f} seconds):")
        for res in results:
            st.write(f"**Title:** {res['title']}")
            st.write(f"**Abstract:** {res['abstract']}")
            st.write("---")

        # Show average query times
        if st.session_state.chroma_query_times:
            avg_chroma = np.mean(st.session_state.chroma_query_times)
        else:
            avg_chroma = None

        if st.session_state.pgvector_query_times:
            avg_pg = np.mean(st.session_state.pgvector_query_times)
        else:
            avg_pg = None

        st.write("## Performance Metrics")
        st.write(f"ChromaDB Average Query Time: {avg_chroma:.4f}s" if avg_chroma is not None else "No ChromaDB queries yet.")
        st.write(f"PGVector Average Query Time: {avg_pg:.4f}s" if avg_pg is not None else "No PGVector queries yet.")

        # Display recall if available
        if st.session_state.recall_chroma or st.session_state.recall_pgvector:
            st.write("## Recall Metrics")
            if st.session_state.recall_chroma:
                avg_recall_chroma = np.mean(st.session_state.recall_chroma)
                st.write(f"ChromaDB Average Recall: {avg_recall_chroma:.4f}")
            else:
                st.write("No ChromaDB recall metrics yet.")
            if st.session_state.recall_pgvector:
                avg_recall_pgvector = np.mean(st.session_state.recall_pgvector)
                st.write(f"PGVector Average Recall: {avg_recall_pgvector:.4f}")
            else:
                st.write("No PGVector recall metrics yet.")
    else:
        st.warning("Please enter a query.")

st.markdown("---")

# Benchmarking Section
st.header("Benchmarking")

benchmark_mode = st.radio("Select Benchmark Mode:", ["Manual Queries", "Upload Query File"])

queries = []

if benchmark_mode == "Manual Queries":
    st.subheader("Enter Multiple Queries")
    num_queries = st.number_input("Number of queries to run:", min_value=1, max_value=1000, value=10)
    for i in range(int(num_queries)):
        q = st.text_input(f"Query {i+1}:", key=f"manual_query_{i}")
        if q:
            queries.append(q)
elif benchmark_mode == "Upload Query File":
    st.subheader("Upload a File with Queries")
    uploaded_file = st.file_uploader("Choose a text file with one query per line", type=["txt"])
    if uploaded_file is not None:
        try:
            stringio = StringIO(uploaded_file.getvalue().decode("utf-8"))
            queries = [line.strip() for line in stringio if line.strip()]
            st.write(f"Loaded {len(queries)} queries.")
        except Exception as e:
            st.error(f"Error reading file: {e}")

if queries:
    run_benchmark = st.button("Run Benchmark")
    
    if run_benchmark:
        # Initialize lists to store latencies and recall (if possible)
        chroma_latencies = []
        pgvector_latencies = []
        # Placeholder for recall metrics
        chroma_recalls = []
        pgvector_recalls = []

        progress_bar = st.progress(0)
        status_text = st.empty()

        for idx, q in enumerate(queries):
            status_text.text(f"Processing query {idx+1}/{len(queries)}: {q[:50]}...")

            q_emb = use_model([q]).numpy()[0]

            # ChromaDB search
            results_chroma, chroma_latency = search_chroma(q_emb, top_k=top_k, coll=chroma_coll)
            chroma_latencies.append(chroma_latency)

            # PGVector search
            results_pgvector, pg_latency = search_pgvector(q_emb, top_k=top_k, conn=pg_conn)
            pgvector_latencies.append(pg_latency)

            progress_bar.progress((idx + 1) / len(queries))

        progress_bar.empty()
        status_text.text("Benchmark completed.")

        # Convert latencies to DataFrame
        df_benchmark = pd.DataFrame({
            'ChromaDB Latency (s)': chroma_latencies,
            'PGVector Latency (s)': pgvector_latencies
        })

        st.subheader("Benchmark Results")

        st.write(df_benchmark.describe())

        # Plotting
        st.subheader("Latency Distribution")

        fig, ax = plt.subplots(figsize=(10, 6))
        sns.kdeplot(df_benchmark['ChromaDB Latency (s)'], label='ChromaDB', shade=True, ax=ax)
        sns.kdeplot(df_benchmark['PGVector Latency (s)'], label='PGVector', shade=True, ax=ax)
        ax.set_xlabel("Latency (seconds)")
        ax.set_title("Latency Distribution for ChromaDB and PGVector")
        ax.legend(title='Database')
        st.pyplot(fig)

        st.subheader("Latency Comparison")

        df_melted = df_benchmark.melt(var_name='Database', value_name='Latency (s)')

        fig2, ax2 = plt.subplots(figsize=(10, 6))
        sns.boxplot(x='Database', y='Latency (s)', data=df_melted, ax=ax2)
        ax2.set_title("Latency Comparison between ChromaDB and PGVector")
        ax2.legend().remove()  # Boxplot has inherent labels
        st.pyplot(fig2)

        st.subheader("Latency Over Queries")

        fig3, ax3 = plt.subplots(figsize=(10, 6))
        ax3.plot(range(1, len(chroma_latencies)+1), chroma_latencies, label='ChromaDB', marker='o')
        ax3.plot(range(1, len(pgvector_latencies)+1), pgvector_latencies, label='PGVector', marker='x')
        ax3.set_xlabel("Query Number")
        ax3.set_ylabel("Latency (seconds)")
        ax3.set_title("Latency Over Queries")
        ax3.legend(title='Database')
        st.pyplot(fig3)

        # Optionally, download the benchmark results
        csv = df_benchmark.to_csv(index=False).encode('utf-8')
        st.download_button(
            label="Download Benchmark Results as CSV",
            data=csv,
            file_name='benchmark_results.csv',
            mime='text/csv',
        )

        # If recall metrics are available, display them
        if chroma_recalls or pgvector_recalls:
            df_recall = pd.DataFrame({
                'ChromaDB Recall': chroma_recalls,
                'PGVector Recall': pgvector_recalls
            })

            st.subheader("Recall Metrics")

            st.write(df_recall.describe())

            # Plot Recall Distribution
            fig_recall, ax_recall = plt.subplots(figsize=(10, 6))
            sns.kdeplot(df_recall['ChromaDB Recall'], label='ChromaDB', shade=True, ax=ax_recall)
            sns.kdeplot(df_recall['PGVector Recall'], label='PGVector', shade=True, ax=ax_recall)
            ax_recall.set_xlabel("Recall")
            ax_recall.set_title("Recall Distribution for ChromaDB and PGVector")
            ax_recall.legend(title='Database')
            st.pyplot(fig_recall)

            # Plot Recall Comparison
            df_recall_melted = df_recall.melt(var_name='Database', value_name='Recall')
            fig_recall_box, ax_recall_box = plt.subplots(figsize=(10, 6))
            sns.boxplot(x='Database', y='Recall', data=df_recall_melted, ax=ax_recall_box)
            ax_recall_box.set_title("Recall Comparison between ChromaDB and PGVector")
            ax_recall_box.legend().remove()  # Boxplot has inherent labels
            st.pyplot(fig_recall_box)

            # Plot Recall Over Queries
            fig_recall_over, ax_recall_over = plt.subplots(figsize=(10, 6))
            ax_recall_over.plot(range(1, len(chroma_recalls)+1), chroma_recalls, label='ChromaDB', marker='o')
            ax_recall_over.plot(range(1, len(pgvector_recalls)+1), pgvector_recalls, label='PGVector', marker='x')
            ax_recall_over.set_xlabel("Query Number")
            ax_recall_over.set_ylabel("Recall")
            ax_recall_over.set_title("Recall Over Queries")
            ax_recall_over.legend(title='Database')
            st.pyplot(fig_recall_over)

    # Optionally, if recall is computed during benchmark, add it here

st.markdown("---")

# Performance Metrics Visualization (optional)
st.header("Performance Metrics Visualization")

if st.session_state.chroma_query_times or st.session_state.pgvector_query_times:
    df_metrics = pd.DataFrame({
        'ChromaDB': st.session_state.chroma_query_times,
        'PGVector': st.session_state.pgvector_query_times
    })

    st.subheader("Historical Query Latencies")

    fig4, ax4 = plt.subplots(figsize=(10, 6))
    sns.lineplot(data=df_metrics, ax=ax4)
    ax4.set_xlabel("Query Number")
    ax4.set_ylabel("Latency (seconds)")
    ax4.set_title("Historical Query Latencies")
    ax4.legend(title='Database')
    st.pyplot(fig4)

    st.subheader("Latency Distribution (Historical)")

    fig5, ax5 = plt.subplots(figsize=(10, 6))
    sns.histplot(df_metrics['ChromaDB'], color='blue', label='ChromaDB', kde=True, stat="density", linewidth=0)
    sns.histplot(df_metrics['PGVector'], color='orange', label='PGVector', kde=True, stat="density", linewidth=0, alpha=0.6)
    ax5.legend(title='Database')
    ax5.set_xlabel("Latency (seconds)")
    ax5.set_title("Latency Distribution (Historical Queries)")
    st.pyplot(fig5)
else:
    st.write("No historical query data to display.")


Writing app.py


## NEXT:
**In terminal, go to the directory where app.py is and run:**
`Streamlit run app.py`